In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

In [2]:
# Clean text
from tqdm import tqdm
tqdm.pandas()
punctuation = ['.', '..', '...', ',', ':', ';', '-', '*', '"', '!', '?']
alphabet = 'abcdefghijklmnopqrstuvwxyz'
def clean_text(x):
    x.lower()
    for p in punctuation:
        x.replace(p, '')
    return x

def extract_features(in_df, train_flag=False):
    df = in_df.copy()
    df['text_cleaned'] = df['text'].apply(lambda x: clean_text(x))
    df['n_.'] = df['text'].str.count('\.')
    df['n_...'] = df['text'].str.count('\...')
    df['n_,'] = df['text'].str.count('\,')
    df['n_:'] = df['text'].str.count('\:')
    df['n_;'] = df['text'].str.count('\;')
    df['n_-'] = df['text'].str.count('\-')
    df['n_?'] = df['text'].str.count('\?')
    df['n_!'] = df['text'].str.count('\!')
    df['n_\''] = df['text'].str.count('\'')
    df['n_"'] = df['text'].str.count('\"')

    # First words in a sentence
    df['n_The '] = df['text'].str.count('The ')
    df['n_I '] = df['text'].str.count('I ')
    df['n_It '] = df['text'].str.count('It ')
    df['n_He '] = df['text'].str.count('He ')
    df['n_Me '] = df['text'].str.count('Me ')
    df['n_She '] = df['text'].str.count('She ')
    df['n_We '] = df['text'].str.count('We ')
    df['n_They '] = df['text'].str.count('They ')
    df['n_You '] = df['text'].str.count('You ')
    df['n_the'] = df['text_cleaned'].str.count('the ')
    df['n_ a '] = df['text_cleaned'].str.count(' a ')
    df['n_appear'] = df['text_cleaned'].str.count('appear')
    df['n_little'] = df['text_cleaned'].str.count('little')
    df['n_was '] = df['text_cleaned'].str.count('was ')
    df['n_one '] = df['text_cleaned'].str.count('one ')
    df['n_two '] = df['text_cleaned'].str.count('two ')
    df['n_three '] = df['text_cleaned'].str.count('three ')
    df['n_ten '] = df['text_cleaned'].str.count('ten ')
    df['n_is '] = df['text_cleaned'].str.count('is ')
    df['n_are '] = df['text_cleaned'].str.count('are ')
    df['n_ed'] = df['text_cleaned'].str.count('ed ')
    df['n_however'] = df['text_cleaned'].str.count('however')
    df['n_ to '] = df['text_cleaned'].str.count(' to ')
    df['n_into'] = df['text_cleaned'].str.count('into')
    df['n_about '] = df['text_cleaned'].str.count('about ')
    df['n_th'] = df['text_cleaned'].str.count('th')
    df['n_er'] = df['text_cleaned'].str.count('er')
    df['n_ex'] = df['text_cleaned'].str.count('ex')
    df['n_an '] = df['text_cleaned'].str.count('an ')
    df['n_ground'] = df['text_cleaned'].str.count('ground')
    df['n_any'] = df['text_cleaned'].str.count('any')
    df['n_silence'] = df['text_cleaned'].str.count('silence')
    df['n_wall'] = df['text_cleaned'].str.count('wall')
    
    # Find numbers of different combinations
    for c in tqdm(alphabet.upper()):
        df['n_' + c] = df['text'].str.count(c)
        df['n_' + c + '.'] = df['text'].str.count(c + '\.')
        df['n_' + c + ','] = df['text'].str.count(c + '\,')

        for c2 in alphabet:
            df['n_' + c + c2] = df['text'].str.count(c + c2)
            df['n_' + c + c2 + '.'] = df['text'].str.count(c + c2 + '\.')
            df['n_' + c + c2 + ','] = df['text'].str.count(c + c2 + '\,')

    for c in tqdm(alphabet):
        df['n_' + c + '.'] = df['text'].str.count(c + '\.')
        df['n_' + c + ','] = df['text'].str.count(c + '\,')
        df['n_' + c + '?'] = df['text'].str.count(c + '\?')
        df['n_' + c + ';'] = df['text'].str.count(c + '\;')
        df['n_' + c + ':'] = df['text'].str.count(c + '\:')

        for c2 in alphabet:
            df['n_' + c + c2 + '.'] = df['text'].str.count(c + c2 + '\.')
            df['n_' + c + c2 + ','] = df['text'].str.count(c + c2 + '\,')
            df['n_' + c + c2 + '?'] = df['text'].str.count(c + c2 + '\?')
            df['n_' + c + c2 + ';'] = df['text'].str.count(c + c2 + '\;')
            df['n_' + c + c2 + ':'] = df['text'].str.count(c + c2 + '\:')
            df['n_' + c + ', ' + c2] = df['text'].str.count(c + '\, ' + c2)

    # And now starting processing of cleaned text
    for c in tqdm(alphabet):
        df['n_' + c] = df['text_cleaned'].str.count(c)
        df['n_' + c + ' '] = df['text_cleaned'].str.count(c + ' ')
        df['n_' + ' ' + c] = df['text_cleaned'].str.count(' ' + c)

        for c2 in alphabet:
            df['n_' + c + c2] = df['text_cleaned'].str.count(c + c2)
            df['n_' + c + c2 + ' '] = df['text_cleaned'].str.count(c + c2 + ' ')
            df['n_' + ' ' + c + c2] = df['text_cleaned'].str.count(' ' + c + c2)
            df['n_' + c + ' ' + c2] = df['text_cleaned'].str.count(c + ' ' + c2)

            for c3 in alphabet:
                df['n_' + c + c2 + c3] = df['text_cleaned'].str.count(c + c2 + c3)
                
    if train_flag:
        df.drop(['text_cleaned','text','author','id'], axis=1, inplace=True)
    else:
        df.drop(['text_cleaned','text','id'], axis=1, inplace=True)
    return df.values
    
print('Processing train...')
train_hand_features = extract_features(train_df,train_flag=True)
print('Processing test...')
test_hand_features = extract_features(test_df)
print(train_hand_features.shape,test_hand_features.shape)

Processing train...


100%|██████████| 26/26 [08:07<00:00, 18.76s/it]


Processing test...


100%|██████████| 26/26 [03:38<00:00,  8.41s/it]


(19579, 26685) (8392, 26685)


In [3]:
def extract_features_2(df):
    df['text_cleaned'] = df['text'].apply(lambda x: clean_text(x))
    df['len'] = df['text'].apply(lambda x: len(x))
    df['n_words'] = df['text'].apply(lambda x: len(x.split(' ')))
    df['n_.'] = df['text'].str.count('\.')
    df['n_...'] = df['text'].str.count('\...')
    df['n_,'] = df['text'].str.count('\,')
    df['n_:'] = df['text'].str.count('\:')
    df['n_;'] = df['text'].str.count('\;')
    df['n_-'] = df['text'].str.count('\-')
    df['n_?'] = df['text'].str.count('\?')
    df['n_!'] = df['text'].str.count('\!')
    df['n_\''] = df['text'].str.count('\'')
    df['n_"'] = df['text'].str.count('\"')

    # First words in a sentence
    df['n_The '] = df['text'].str.count('The ')
    df['n_I '] = df['text'].str.count('I ')
    df['n_It '] = df['text'].str.count('It ')
    df['n_He '] = df['text'].str.count('He ')
    df['n_Me '] = df['text'].str.count('Me ')
    df['n_She '] = df['text'].str.count('She ')
    df['n_We '] = df['text'].str.count('We ')
    df['n_They '] = df['text'].str.count('They ')
    df['n_You '] = df['text'].str.count('You ')
    df['n_the'] = df['text_cleaned'].str.count('the ')
    df['n_ a '] = df['text_cleaned'].str.count(' a ')
    df['n_appear'] = df['text_cleaned'].str.count('appear')
    df['n_little'] = df['text_cleaned'].str.count('little')
    df['n_was '] = df['text_cleaned'].str.count('was ')
    df['n_one '] = df['text_cleaned'].str.count('one ')
    df['n_two '] = df['text_cleaned'].str.count('two ')
    df['n_three '] = df['text_cleaned'].str.count('three ')
    df['n_ten '] = df['text_cleaned'].str.count('ten ')
    df['n_is '] = df['text_cleaned'].str.count('is ')
    df['n_are '] = df['text_cleaned'].str.count('are ')
    df['n_ed'] = df['text_cleaned'].str.count('ed ')
    df['n_however'] = df['text_cleaned'].str.count('however')
    df['n_ to '] = df['text_cleaned'].str.count(' to ')
    df['n_into'] = df['text_cleaned'].str.count('into')
    df['n_about '] = df['text_cleaned'].str.count('about ')
    df['n_th'] = df['text_cleaned'].str.count('th')
    df['n_er'] = df['text_cleaned'].str.count('er')
    df['n_ex'] = df['text_cleaned'].str.count('ex')
    df['n_an '] = df['text_cleaned'].str.count('an ')
    df['n_ground'] = df['text_cleaned'].str.count('ground')
    df['n_any'] = df['text_cleaned'].str.count('any')
    df['n_silence'] = df['text_cleaned'].str.count('silence')
    df['n_wall'] = df['text_cleaned'].str.count('wall')

    df.drop(['text_cleaned'], axis=1, inplace=True)

print('Processing train...')
extract_features_2(train_df)
print('Processing test...')
extract_features_2(test_df)

Processing train...
Processing test...


In [4]:
# https://spacy.io/usage/models#usage-import
# https://spacy.io/usage/models
import en_core_web_sm
spacy_nlp = en_core_web_sm.load()

# change ne to tag
def get_spacy_text(s):
    pos,tag,dep = '','',''
    for token in spacy_nlp(s):
        pos = pos + ' ' + token.pos_
        tag = tag + ' ' + token.tag_
        dep = dep + ' ' + token.dep_

    return pos,tag,dep

print(get_spacy_text('this is kaggle spooky games.'))

import time
start_t = time.time()
poss,tags,deps = [],[],[]
for s in train_df["text"].values:
    pos,tag,dep = get_spacy_text(s)
    poss.append(pos)
    tags.append(tag)
    deps.append(dep)
train_df['pos_txt'],train_df['tag_txt'],train_df['dep_txt'] = poss, tags, deps
print('train done',time.time() - start_t)


start_t = time.time()
poss,tags,deps = [],[],[]
for s in test_df["text"].values:
    pos,tag,dep = get_spacy_text(s)
    poss.append(pos)
    tags.append(tag)
    deps.append(dep)
test_df['pos_txt'],test_df['tag_txt'],test_df['dep_txt'] = poss, tags, deps
print('test done', time.time() - start_t)

(' DET VERB NOUN ADJ NOUN PUNCT', ' DT VBZ NN JJ NNS .', ' nsubj ROOT nmod amod attr punct')
train done 289.98655915260315
test done 134.5491213798523


In [5]:
# cnt on tag
c_vec3 = CountVectorizer(lowercase=False,ngram_range=(1,1))
c_vec3.fit(train_df['tag_txt'].values.tolist() + test_df['tag_txt'].values.tolist())
train_cvec3 = c_vec3.transform(train_df['tag_txt'].values.tolist()).toarray()
test_cvec3 = c_vec3.transform(test_df['tag_txt'].values.tolist()).toarray()
print(train_cvec3.shape,test_cvec3.shape)

# cnt on ne
c_vec4 = CountVectorizer(lowercase=False,ngram_range=(1,2))
c_vec4.fit(train_df['pos_txt'].values.tolist() + test_df['pos_txt'].values.tolist())
train_cvec4 = c_vec4.transform(train_df['pos_txt'].values.tolist()).toarray()
test_cvec4 = c_vec4.transform(test_df['pos_txt'].values.tolist()).toarray()
print(train_cvec4.shape,test_cvec4.shape)

# cnt on dep
c_vec7 = CountVectorizer(lowercase=False,ngram_range=(1,1))
c_vec7.fit(train_df['dep_txt'].values.tolist() + test_df['dep_txt'].values.tolist())
train_cvec7 = c_vec7.transform(train_df['dep_txt'].values.tolist()).toarray()
test_cvec7 = c_vec7.transform(test_df['dep_txt'].values.tolist()).toarray()
print(train_cvec7.shape,test_cvec7.shape)

# tfidf on tag
tf_vec5 = TfidfVectorizer(lowercase=False,ngram_range=(1,1))
tf_vec5.fit(train_df['tag_txt'].values.tolist() + test_df['tag_txt'].values.tolist())
train_tf5 = tf_vec5.transform(train_df['tag_txt'].values.tolist()).toarray()
test_tf5 = tf_vec5.transform(test_df['tag_txt'].values.tolist()).toarray()
print(train_tf5.shape,test_tf5.shape)

# tfidf on ne
tf_vec6 = TfidfVectorizer(lowercase=False,ngram_range=(1,2))
tf_vec6.fit(train_df['pos_txt'].values.tolist() + test_df['pos_txt'].values.tolist())
train_tf6 = tf_vec6.transform(train_df['pos_txt'].values.tolist()).toarray()
test_tf6 = tf_vec6.transform(test_df['pos_txt'].values.tolist()).toarray()
print(train_tf6.shape,test_tf6.shape)

# tfidf on dep
tf_vec8 = TfidfVectorizer(lowercase=False,ngram_range=(1,1))
tf_vec8.fit(train_df['dep_txt'].values.tolist() + test_df['dep_txt'].values.tolist())
train_tf8 = tf_vec8.transform(train_df['dep_txt'].values.tolist()).toarray()
test_tf8 = tf_vec8.transform(test_df['dep_txt'].values.tolist()).toarray()
print(train_tf8.shape,test_tf8.shape)

(19579, 38) (8392, 38)
(19579, 186) (8392, 186)
(19579, 45) (8392, 45)
(19579, 38) (8392, 38)
(19579, 186) (8392, 186)
(19579, 45) (8392, 45)


In [6]:
all_nlp_train = np.hstack([train_cvec3,train_cvec4,train_tf5,train_tf6,train_cvec7, train_tf8]) 
all_nlp_test = np.hstack([test_cvec3,test_cvec4,test_tf5,test_tf6, test_cvec7, test_tf8]) 
print('nlp feat done')

nlp feat done


In [7]:
# replace
# train_df['text'] = train_df['text'].str.replace('[^a-zA-Z0-9]', ' ')
# test_df['text'] =test_df['text'].str.replace('[^a-zA-Z0-9]', ' ')

## Number of words in the text ##
train_df["num_words"] = train_df["text"].apply(lambda x: len(str(x).split()))
test_df["num_words"] = test_df["text"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train_df["num_unique_words"] = train_df["text"].apply(lambda x: len(set(str(x).split())))
test_df["num_unique_words"] = test_df["text"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
train_df["num_chars"] = train_df["text"].apply(lambda x: len(str(x)))
test_df["num_chars"] = test_df["text"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
eng_stopwords = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]
train_df["num_stopwords"] = train_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_df["num_stopwords"] = test_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
import string
train_df["num_punctuations"] =train_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test_df["num_punctuations"] =test_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
train_df["num_words_upper"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_df["num_words_upper"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
train_df["num_words_title"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test_df["num_words_title"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
train_df["mean_word_len"] = train_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df["mean_word_len"] = test_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [8]:
## Prepare the data for modeling ###
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)
train_id = train_df['id'].values
test_id = test_df['id'].values

## add tfidf and svd
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), max_df=0.8,lowercase=False, sublinear_tf=True)
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

print(train_tfidf.shape,test_tfidf.shape)

# svd1
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
print(train_svd.shape,test_svd.shape)

## add tfidf char
tfidf_vec = TfidfVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8, sublinear_tf=True)
full_tfidf2 = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf2 = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf2 = tfidf_vec.transform(test_df['text'].values.tolist())
print(train_tfidf2.shape,test_tfidf2.shape)

## add svd2
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf2)
train_svd2 = pd.DataFrame(svd_obj.transform(train_tfidf2))
test_svd2 = pd.DataFrame(svd_obj.transform(test_tfidf2))
print(train_svd2.shape,test_svd2.shape)


## add cnt vec
c_vec = CountVectorizer(ngram_range=(1,3),max_df=0.8, lowercase=False)
full_cvec1 = c_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec = c_vec.transform(train_df['text'].values.tolist())
test_cvec = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec.shape,test_cvec.shape)

## add svd3
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_cvec1)
train_svd3 = pd.DataFrame(svd_obj.transform(train_cvec))
test_svd3 = pd.DataFrame(svd_obj.transform(test_cvec))

# add cnt char
c_vec = CountVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8)
full_cvec2 = c_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec2 = c_vec.transform(train_df['text'].values.tolist())
test_cvec2 = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec2.shape,test_cvec2.shape)

## add svd4
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_cvec2)
train_svd4 = pd.DataFrame(svd_obj.transform(train_cvec2))
test_svd4 = pd.DataFrame(svd_obj.transform(test_cvec2))

all_svd_train = np.hstack([train_svd,train_svd2,train_svd3,train_svd4])
all_svd_test = np.hstack([test_svd,test_svd2,test_svd3,test_svd4])

(19579, 885129) (8392, 885129)
(19579, 30) (8392, 30)
(19579, 1354551) (8392, 1354551)
(19579, 30) (8392, 30)
(19579, 885129) (8392, 885129)
(19579, 1354551) (8392, 1354551)


In [9]:
# add naive feature
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

feat_cnt = 5
train_Y = train_y

def gen_nb_feats(rnd=1):
    help_tfidf_train,help_tfidf_test = np.zeros((19579,3)),np.zeros((8392,3))
    help_tfidf_train2,help_tfidf_test2 = np.zeros((19579,3)),np.zeros((8392,3))
    help_cnt1_train,help_cnt1_test = np.zeros((19579,3)),np.zeros((8392,3))
    help_cnt2_train,help_cnt2_test = np.zeros((19579,3)),np.zeros((8392,3))
    hand_train, hand_test = np.zeros((19579,3)),np.zeros((8392,3))

    kf = KFold(n_splits=feat_cnt, shuffle=True, random_state=23*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        # tfidf to nb
        X_train, X_test = train_tfidf[train_index], train_tfidf[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        tmp_model = MultinomialNB(alpha=0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_tfidf)
        help_tfidf_train[test_index] = tmp_train_feat
        help_tfidf_test += tmp_test_feat/feat_cnt

        # tfidf to nb
        X_train, X_test = train_tfidf2[train_index], train_tfidf2[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_tfidf2)
        help_tfidf_train2[test_index] = tmp_train_feat
        help_tfidf_test2 += tmp_test_feat/feat_cnt

        # count vec to nb
        X_train, X_test = train_cvec[train_index], train_cvec[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_cvec)
        help_cnt1_train[test_index] = tmp_train_feat
        help_cnt1_test += tmp_test_feat/feat_cnt

        # count vec2 to nb 
        X_train, X_test = train_cvec2[train_index], train_cvec2[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_cvec2)
        help_cnt2_train[test_index] = tmp_train_feat
        help_cnt2_test += tmp_test_feat/feat_cnt
        
        # hand feature to nb
        X_train, X_test = train_hand_features[train_index], train_hand_features[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_hand_features)
        hand_train[test_index] = tmp_train_feat
        hand_test += tmp_test_feat/feat_cnt
    
    help_train_feat = np.hstack([help_tfidf_train,help_tfidf_train2,help_cnt1_train,help_cnt2_train,hand_train])
    help_test_feat = np.hstack([help_tfidf_test,help_tfidf_test2,help_cnt1_test,help_cnt2_test,hand_test])

    return help_train_feat,help_test_feat

help_train_feat,help_test_feat = gen_nb_feats(1)
print(help_train_feat.shape,help_test_feat.shape)
help_train_feat2,help_test_feat2 = gen_nb_feats(2)
help_train_feat3,help_test_feat3 = gen_nb_feats(3)

(19579, 15) (8392, 15)


In [10]:
# add cnn feat
from keras.layers import Embedding, CuDNNLSTM, Dense, Flatten, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.metrics import log_loss
import gc
print('import keras done')

Using TensorFlow backend.


import keras done


In [11]:
def get_cnn_feats(rnd=1):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    best_val_train_pred, best_val_test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 30000
    N = 10
    MAX_LEN = 150
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=233*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(Conv1D(16,
                         3,
                         padding='valid',
                         activation='relu',
                         strides=1))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.1,
                  batch_size=64, epochs=10, 
                  verbose=2,
                  callbacks=[model_chk],
                  shuffle=False
                 )
 
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        
        # best val model
        model = load_model(MODEL_P)
        best_val_train_pred[test_index] = model.predict(ttrain_x[test_index])
        best_val_test_pred += model.predict(ttest_x)/feat_cnt
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred,test_pred,best_val_train_pred,best_val_test_pred

print('def cnn done')

cnn_train1,cnn_test1,cnn_train2,cnn_test2 = get_cnn_feats(1)
cnn_train3,cnn_test3,cnn_train4,cnn_test4 = get_cnn_feats(2)
cnn_train5,cnn_test5,cnn_train6,cnn_test6 = get_cnn_feats(3)

def cnn done
Train on 14096 samples, validate on 1567 samples
Epoch 1/10
Epoch 00001: val_loss improved from inf to 1.06339, saving model to /tmp/nn_model.h5
 - 3s - loss: 1.0855 - acc: 0.4022 - val_loss: 1.0634 - val_acc: 0.4110
Epoch 2/10
Epoch 00002: val_loss improved from 1.06339 to 0.84664, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.9634 - acc: 0.5223 - val_loss: 0.8466 - val_acc: 0.6720
Epoch 3/10
Epoch 00003: val_loss improved from 0.84664 to 0.60759, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.6816 - acc: 0.7548 - val_loss: 0.6076 - val_acc: 0.7703
Epoch 4/10
Epoch 00004: val_loss improved from 0.60759 to 0.48770, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.4627 - acc: 0.8370 - val_loss: 0.4877 - val_acc: 0.8149
Epoch 5/10
Epoch 00005: val_loss improved from 0.48770 to 0.44639, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.3279 - acc: 0.8878 - val_loss: 0.4464 - val_acc: 0.8341
Epoch 6/10
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.2460 - acc: 0.921

Epoch 4/10
Epoch 00004: val_loss improved from 0.56766 to 0.45039, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.4181 - acc: 0.8571 - val_loss: 0.4504 - val_acc: 0.8277
Epoch 5/10
Epoch 00005: val_loss improved from 0.45039 to 0.41223, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2906 - acc: 0.9032 - val_loss: 0.4122 - val_acc: 0.8373
Epoch 6/10
Epoch 00006: val_loss improved from 0.41223 to 0.40640, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2175 - acc: 0.9281 - val_loss: 0.4064 - val_acc: 0.8392
Epoch 7/10
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.1655 - acc: 0.9515 - val_loss: 0.4195 - val_acc: 0.8411
Epoch 8/10
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.1296 - acc: 0.9615 - val_loss: 0.4366 - val_acc: 0.8392
Epoch 9/10
Epoch 00009: val_loss did not improve
 - 1s - loss: 0.1073 - acc: 0.9663 - val_loss: 0.4627 - val_acc: 0.8354
Epoch 10/10
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.0890 - acc: 0.9734 - val_loss: 0.4807 - val_acc: 0.8322

Epoch 7/10
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.1556 - acc: 0.9517 - val_loss: 0.4072 - val_acc: 0.8494
Epoch 8/10
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.1277 - acc: 0.9625 - val_loss: 0.4254 - val_acc: 0.8449
Epoch 9/10
Epoch 00009: val_loss did not improve
 - 1s - loss: 0.1033 - acc: 0.9679 - val_loss: 0.4512 - val_acc: 0.8449
Epoch 10/10
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.0871 - acc: 0.9755 - val_loss: 0.4887 - val_acc: 0.8424
------------------
Train on 14096 samples, validate on 1567 samples
Epoch 1/10
Epoch 00001: val_loss improved from inf to 1.04813, saving model to /tmp/nn_model.h5
 - 2s - loss: 1.0824 - acc: 0.4105 - val_loss: 1.0481 - val_acc: 0.4703
Epoch 2/10
Epoch 00002: val_loss improved from 1.04813 to 0.72585, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.8985 - acc: 0.6051 - val_loss: 0.7258 - val_acc: 0.7364
Epoch 3/10
Epoch 00003: val_loss improved from 0.72585 to 0.49543, saving model to /tmp/nn_model.h5
 - 1s -

In [12]:
# add lstm feat
def get_lstm_feats(rnd=1):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    best_val_train_pred, best_val_test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 16000
    N = 12
    MAX_LEN = 300
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=2333*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(LSTM(N, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(NUM_CLASSES, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.1,
                  batch_size=128, epochs=6, 
                  verbose=2,
                  callbacks=[model_chk],
                  shuffle=False
                 )
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        
        # best val model
        model = load_model(MODEL_P)
        best_val_train_pred[test_index] = model.predict(ttrain_x[test_index])
        best_val_test_pred += model.predict(ttest_x)/feat_cnt
        
        del model
        gc.collect()
        print('------------------')
        
    return train_pred,test_pred,best_val_train_pred,best_val_test_pred

print('def lstm done')
lstm_train1,lstm_test1,lstm_train2,lstm_test2 = get_lstm_feats(1)

def lstm done
Train on 14096 samples, validate on 1567 samples
Epoch 1/6
Epoch 00001: val_loss improved from inf to 0.80975, saving model to /tmp/nn_model.h5
 - 42s - loss: 1.0312 - acc: 0.4702 - val_loss: 0.8098 - val_acc: 0.6662
Epoch 2/6
Epoch 00002: val_loss improved from 0.80975 to 0.49086, saving model to /tmp/nn_model.h5
 - 40s - loss: 0.5627 - acc: 0.7726 - val_loss: 0.4909 - val_acc: 0.8054
Epoch 3/6
Epoch 00003: val_loss improved from 0.49086 to 0.43985, saving model to /tmp/nn_model.h5
 - 40s - loss: 0.3129 - acc: 0.8851 - val_loss: 0.4399 - val_acc: 0.8271
Epoch 4/6
Epoch 00004: val_loss did not improve
 - 40s - loss: 0.2018 - acc: 0.9296 - val_loss: 0.4679 - val_acc: 0.8398
Epoch 5/6
Epoch 00005: val_loss did not improve
 - 39s - loss: 0.1430 - acc: 0.9496 - val_loss: 0.4951 - val_acc: 0.8334
Epoch 6/6
Epoch 00006: val_loss did not improve
 - 39s - loss: 0.1059 - acc: 0.9640 - val_loss: 0.5373 - val_acc: 0.8411
------------------
Train on 14096 samples, validate on 1567 sa

In [13]:
def get_nn_feats(rnd=1):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    best_val_train_pred, best_val_test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 30000
    N = 10
    MAX_LEN = 100
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=233*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(30, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.3,
                  batch_size=64, epochs=20, 
                  verbose=2,
                  callbacks=[model_chk],
                  shuffle=False
                 )
 
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        
        # best val model
        model = load_model(MODEL_P)
        best_val_train_pred[test_index] = model.predict(ttrain_x[test_index])
        best_val_test_pred += model.predict(ttest_x)/feat_cnt
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred,test_pred,best_val_train_pred,best_val_test_pred

print('def cnn done')

nn_train1,nn_test1,nn_train2,nn_test2 = get_nn_feats(4)
nn_train3,nn_test3,nn_train4,nn_test4 = get_nn_feats(5)
nn_train5,nn_test5,nn_train6,nn_test6 = get_nn_feats(6)


all_nn_train = np.hstack([lstm_train1, lstm_train2, 
                          cnn_train1, cnn_train2,cnn_train3, cnn_train4,cnn_train5, cnn_train6,
                          nn_train1,nn_train2,nn_train3,nn_train4,nn_train5,nn_train6
                         ])
all_nn_test = np.hstack([lstm_test1, lstm_test2, 
                         cnn_test1, cnn_test2,cnn_test3, cnn_test4,cnn_test5, cnn_test6,
                         nn_test1,nn_test2,nn_test3,nn_test4,nn_test5,nn_test6
])

def cnn done
Train on 10964 samples, validate on 4699 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 1.07740, saving model to /tmp/nn_model.h5
 - 4s - loss: 1.0881 - acc: 0.3928 - val_loss: 1.0774 - val_acc: 0.4035
Epoch 2/20
Epoch 00002: val_loss improved from 1.07740 to 0.99502, saving model to /tmp/nn_model.h5
 - 1s - loss: 1.0476 - acc: 0.4467 - val_loss: 0.9950 - val_acc: 0.5471
Epoch 3/20
Epoch 00003: val_loss improved from 0.99502 to 0.79465, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.8821 - acc: 0.6530 - val_loss: 0.7946 - val_acc: 0.7004
Epoch 4/20
Epoch 00004: val_loss improved from 0.79465 to 0.64252, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.6575 - acc: 0.7758 - val_loss: 0.6425 - val_acc: 0.7531
Epoch 5/20
Epoch 00005: val_loss improved from 0.64252 to 0.55364, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.5029 - acc: 0.8329 - val_loss: 0.5536 - val_acc: 0.7855
Epoch 6/20
Epoch 00006: val_loss improved from 0.55364 to 0.50103, saving model to

Epoch 13/20
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.1275 - acc: 0.9668 - val_loss: 0.4349 - val_acc: 0.8332
Epoch 14/20
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.1118 - acc: 0.9710 - val_loss: 0.4410 - val_acc: 0.8319
Epoch 15/20
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.0985 - acc: 0.9754 - val_loss: 0.4524 - val_acc: 0.8302
Epoch 16/20
Epoch 00016: val_loss did not improve
 - 1s - loss: 0.0865 - acc: 0.9774 - val_loss: 0.4611 - val_acc: 0.8298
Epoch 17/20
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.0761 - acc: 0.9819 - val_loss: 0.4722 - val_acc: 0.8283
Epoch 18/20
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.0675 - acc: 0.9848 - val_loss: 0.4832 - val_acc: 0.8276
Epoch 19/20
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.0599 - acc: 0.9870 - val_loss: 0.4984 - val_acc: 0.8268
Epoch 20/20
Epoch 00020: val_loss did not improve
 - 1s - loss: 0.0523 - acc: 0.9881 - val_loss: 0.5065 - val_acc: 0.8253
------------------
Train

Epoch 00007: val_loss improved from 0.50757 to 0.47509, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.3385 - acc: 0.8933 - val_loss: 0.4751 - val_acc: 0.8070
Epoch 8/20
Epoch 00008: val_loss improved from 0.47509 to 0.45422, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2809 - acc: 0.9128 - val_loss: 0.4542 - val_acc: 0.8161
Epoch 9/20
Epoch 00009: val_loss improved from 0.45422 to 0.43939, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2359 - acc: 0.9304 - val_loss: 0.4394 - val_acc: 0.8221
Epoch 10/20
Epoch 00010: val_loss improved from 0.43939 to 0.43597, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2011 - acc: 0.9412 - val_loss: 0.4360 - val_acc: 0.8266
Epoch 11/20
Epoch 00011: val_loss improved from 0.43597 to 0.43302, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.1731 - acc: 0.9507 - val_loss: 0.4330 - val_acc: 0.8293
Epoch 12/20
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.1497 - acc: 0.9578 - val_loss: 0.4386 - val_acc: 0.8280
Epoch 13/20
Epoch 00013: va

 - 4s - loss: 1.0853 - acc: 0.4052 - val_loss: 1.0737 - val_acc: 0.4126
Epoch 2/20
Epoch 00002: val_loss improved from 1.07374 to 0.99581, saving model to /tmp/nn_model.h5
 - 1s - loss: 1.0464 - acc: 0.4422 - val_loss: 0.9958 - val_acc: 0.5103
Epoch 3/20
Epoch 00003: val_loss improved from 0.99581 to 0.81888, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.9004 - acc: 0.6024 - val_loss: 0.8189 - val_acc: 0.6927
Epoch 4/20
Epoch 00004: val_loss improved from 0.81888 to 0.66223, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.6955 - acc: 0.7593 - val_loss: 0.6622 - val_acc: 0.7680
Epoch 5/20
Epoch 00005: val_loss improved from 0.66223 to 0.56755, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.5343 - acc: 0.8252 - val_loss: 0.5675 - val_acc: 0.7883
Epoch 6/20
Epoch 00006: val_loss improved from 0.56755 to 0.51337, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.4251 - acc: 0.8627 - val_loss: 0.5134 - val_acc: 0.7978
Epoch 7/20
Epoch 00007: val_loss improved from 0.51337 to 0.48147, s

Epoch 15/20
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.0995 - acc: 0.9755 - val_loss: 0.4390 - val_acc: 0.8300
Epoch 16/20
Epoch 00016: val_loss did not improve
 - 1s - loss: 0.0880 - acc: 0.9794 - val_loss: 0.4486 - val_acc: 0.8310
Epoch 17/20
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.0786 - acc: 0.9814 - val_loss: 0.4582 - val_acc: 0.8308
Epoch 18/20
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.0684 - acc: 0.9849 - val_loss: 0.4652 - val_acc: 0.8319
Epoch 19/20
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.0611 - acc: 0.9867 - val_loss: 0.4800 - val_acc: 0.8278
Epoch 20/20
Epoch 00020: val_loss did not improve
 - 1s - loss: 0.0540 - acc: 0.9870 - val_loss: 0.4971 - val_acc: 0.8278
------------------
Train on 10964 samples, validate on 4699 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 1.07565, saving model to /tmp/nn_model.h5
 - 5s - loss: 1.0850 - acc: 0.4062 - val_loss: 1.0757 - val_acc: 0.4046
Epoch 2/20
Epoch 00002: val_loss

Epoch 9/20
Epoch 00009: val_loss improved from 0.47007 to 0.44915, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2509 - acc: 0.9226 - val_loss: 0.4491 - val_acc: 0.8215
Epoch 10/20
Epoch 00010: val_loss improved from 0.44915 to 0.44592, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2152 - acc: 0.9373 - val_loss: 0.4459 - val_acc: 0.8234
Epoch 11/20
Epoch 00011: val_loss improved from 0.44592 to 0.44066, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.1853 - acc: 0.9450 - val_loss: 0.4407 - val_acc: 0.8270
Epoch 12/20
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.1601 - acc: 0.9556 - val_loss: 0.4418 - val_acc: 0.8289
Epoch 13/20
Epoch 00013: val_loss improved from 0.44066 to 0.43354, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.1388 - acc: 0.9617 - val_loss: 0.4335 - val_acc: 0.8315
Epoch 14/20
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.1206 - acc: 0.9675 - val_loss: 0.4426 - val_acc: 0.8291
Epoch 15/20
Epoch 00015: val_loss did not improve
 - 1s - loss: 0

In [14]:
# combine feats
cols_to_drop = ['id', 'text','tag_txt','pos_txt','dep_txt']
train_X = train_df.drop(cols_to_drop+['author'], axis=1).values
test_X = test_df.drop(cols_to_drop, axis=1).values
train_X = np.hstack([train_X, all_svd_train, all_nlp_train])
test_X = np.hstack([test_X, all_svd_test, all_nlp_test])

f_train_X = np.hstack([train_X, help_train_feat,help_train_feat2,help_train_feat3,all_nn_train])
#f_train_X = np.round(f_train_X,4)
f_test_X = np.hstack([test_X, help_test_feat,help_test_feat2,help_test_feat3,all_nn_test])
#f_test_X = np.round(f_test_X,4)
print(f_train_X.shape, f_test_X.shape)

(19579, 798) (8392, 798)


In [15]:
import pickle
with open('feat.pkl','wb') as fout:
    pickle.dump([f_train_X,f_test_X],fout)
print('dump for xgb')

dump for xgb


In [25]:
from sklearn.model_selection import StratifiedKFold
def cv_test(k_cnt=3, s_flag = False):
    rnd = 42
    if s_flag:
        kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    else:
        kf = KFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    test_pred = None
    weighted_test_pred = None
    org_train_pred = None
    avg_k_score = 0
    reverse_score = 0
    best_loss = 100
    best_single_pred = None
    for train_index, test_index in kf.split(f_train_X,train_Y):
        X_train, X_test = f_train_X[train_index], f_train_X[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 3,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':3,
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(f_test_X)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=200)
        
        # get res
        train_pred = m.predict(d_train)
        valid_pred = m.predict(d_valid)
        tmp_train_pred = m.predict(xgb.DMatrix(f_train_X))
        
        # cal score
        train_score = log_loss(y_train,train_pred)
        valid_score = log_loss(y_test,valid_pred)
        print('train log loss',train_score,'valid log loss',valid_score)
        avg_k_score += valid_score
        rev_valid_score = 1.0/valid_score # use for weighted
        reverse_score += rev_valid_score # sum
        print('rev',rev_valid_score)
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            weighted_test_pred = test_pred*rev_valid_score
            org_train_pred = tmp_train_pred
            best_loss = valid_score
            best_single_pred = test_pred
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            weighted_test_pred += curr_pred*rev_valid_score # fix bug here
            org_train_pred += tmp_train_pred
            # find better single model
            if valid_score < best_loss:
                print('BETTER')
                best_loss = valid_score
                best_single_pred = curr_pred

    # avg
    test_pred = test_pred / k_cnt
    test_pred = np.round(test_pred,4)
    org_train_pred = org_train_pred / k_cnt
    avg_k_score = avg_k_score/k_cnt

    submiss=pd.read_csv("./input/sample_submission.csv")
    submiss['EAP']=test_pred[:,0]
    submiss['HPL']=test_pred[:,1]
    submiss['MWS']=test_pred[:,2]
    submiss.to_csv("results/xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('--------------')
    print(reverse_score)
    # weigthed
    submiss=pd.read_csv("./input/sample_submission.csv")
    weighted_test_pred = weighted_test_pred / reverse_score
    weighted_test_pred = np.round(weighted_test_pred,4)
    submiss['EAP']=weighted_test_pred[:,0]
    submiss['HPL']=weighted_test_pred[:,1]
    submiss['MWS']=weighted_test_pred[:,2]
    submiss.to_csv("results/weighted_xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('---------------')
    # best single
    submiss=pd.read_csv("./input/sample_submission.csv")
    weighted_test_pred = np.round(best_single_pred,4)
    submiss['EAP']=weighted_test_pred[:,0]
    submiss['HPL']=weighted_test_pred[:,1]
    submiss['MWS']=weighted_test_pred[:,2]
    submiss.to_csv("results/single_xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('---------------')
    
    # train log loss
    print('local average valid loss',avg_k_score)
    print('train log loss', log_loss(train_Y,org_train_pred))
    
print('def done')

def done


In [26]:
cv_test(5, True)

[0]	train-mlogloss:1.05485	valid-mlogloss:1.05599
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.221488	valid-mlogloss:0.280795
[400]	train-mlogloss:0.177581	valid-mlogloss:0.270376
Stopping. Best iteration:
[533]	train-mlogloss:0.156503	valid-mlogloss:0.268771

train log loss 0.149523098456 valid log loss 0.269121766835
rev 3.71579011152
[0]	train-mlogloss:1.05491	valid-mlogloss:1.05563
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.222293	valid-mlogloss:0.272717
[400]	train-mlogloss:0.178213	valid-mlogloss:0.264969
Stopping. Best iteration:
[537]	train-mlogloss:0.157412	valid-mlogloss:0.264233

train log loss 0.150395820849 valid log loss 0.264380298594
rev 3.78243010284
BETTER
[0]	train-mlogloss:1.05544	valid-mlogloss:1.055
Multi

In [27]:
cv_test(10, True)
# 276xx not good

[0]	train-mlogloss:1.05503	valid-mlogloss:1.05568
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.227454	valid-mlogloss:0.269699
[400]	train-mlogloss:0.186528	valid-mlogloss:0.257652
[600]	train-mlogloss:0.157903	valid-mlogloss:0.255313
Stopping. Best iteration:
[699]	train-mlogloss:0.146044	valid-mlogloss:0.25433

train log loss 0.140450477448 valid log loss 0.254583709453
rev 3.92798110354
[0]	train-mlogloss:1.05501	valid-mlogloss:1.05633
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.22469	valid-mlogloss:0.294069
[400]	train-mlogloss:0.182786	valid-mlogloss:0.284528
[600]	train-mlogloss:0.154454	valid-mlogloss:0.28286
Stopping. Best iteration:
[559]	train-mlogloss:0.159669	valid-mlogloss:0.282648

train log loss 0.153284262756 val